In [9]:
import pandas as pd
import numpy as np
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, roc_curve, precision_recall_curve, auc
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore")


In [10]:
def create_features(data):
    data = data.copy()
    data['pernah_dihubungi_sebelumnya'] = np.where(
        data['hari_sejak_kontak_sebelumnya'] == 999, 0, 1)
    data['rasio_sukses_sebelumnya'] = (
        data['hasil_kampanye_sebelumnya'].map({'success': 1, 'failure': 0, 'nonexistent': 0})
        * (data['jumlah_kontak_sebelumnya'] + 1)
    )
    data['risk_score'] = (
        0.4 * data['indeks_harga_konsumen'] +
        0.3 * data['suku_bunga_euribor_3bln'] +
        0.3 * data['tingkat_variasi_pekerjaan']
    )
    data['usia_group'] = pd.cut(
        data['usia'], bins=[18, 30, 45, 60, 100],
        labels=['muda', 'paruhbaya', 'senior', 'lansia']
    )
    data['beban_pinjaman'] = (
        data['pinjaman_rumah'].map({'yes': 1, 'no': 0, 'unknown': 0}) +
        data['pinjaman_pribadi'].map({'yes': 1, 'no': 0, 'unknown': 0})
    )
    return data


In [11]:
df = pd.read_csv("training_dataset.csv")
df = create_features(df)

target = "berlangganan_deposito"
features = [col for col in df.columns if col != target and col != "customer_number"]

X = df[features]
y = df[target]

NUM_FEATURES = ['usia', 'jumlah_kontak_kampanye_ini', 'indeks_harga_konsumen',
                'suku_bunga_euribor_3bln', 'rasio_sukses_sebelumnya', 'risk_score']
CAT_FEATURES = ['pekerjaan', 'status_perkawinan', 'pendidikan', 'jenis_kontak',
                'pulau', 'usia_group', 'pernah_dihubungi_sebelumnya']

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), NUM_FEATURES),
    ("cat", OneHotEncoder(handle_unknown="ignore"), CAT_FEATURES)
])


In [12]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int("n_estimators", 100, 500),
        'max_depth': trial.suggest_int("max_depth", 3, 10),
        'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.2),
        'subsample': trial.suggest_float("subsample", 0.5, 1.0),
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.5, 1.0),
        'scale_pos_weight': trial.suggest_float("scale_pos_weight", 1, 10),
        'random_state': 42,
        'tree_method': 'hist',
        'use_label_encoder': False
    }

    model = ImbPipeline([
        ("preprocess", preprocessor),
        ("smote", SMOTE(random_state=42)),
        ("clf", XGBClassifier(**params))
    ])

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X, y, scoring="roc_auc", cv=cv, n_jobs=-1).mean()
    return score


In [13]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

print("Best ROC-AUC:", study.best_value)
print("Best Parameters:", study.best_params)


[I 2025-05-23 14:46:20,356] A new study created in memory with name: no-name-eb95db61-ea0c-4ae7-8e5b-7e968af06104


[I 2025-05-23 14:47:15,333] Trial 0 finished with value: 0.7635399567078124 and parameters: {'n_estimators': 388, 'max_depth': 9, 'learning_rate': 0.07039898808024145, 'subsample': 0.6685527586024014, 'colsample_bytree': 0.5546398026108869, 'scale_pos_weight': 6.57214872638162}. Best is trial 0 with value: 0.7635399567078124.
[I 2025-05-23 14:47:28,520] Trial 1 finished with value: 0.7882299249304652 and parameters: {'n_estimators': 235, 'max_depth': 5, 'learning_rate': 0.03771216438443701, 'subsample': 0.6484977304483833, 'colsample_bytree': 0.8874996982688341, 'scale_pos_weight': 1.8808299207300545}. Best is trial 1 with value: 0.7882299249304652.
[I 2025-05-23 14:47:46,907] Trial 2 finished with value: 0.7788732394217561 and parameters: {'n_estimators': 430, 'max_depth': 5, 'learning_rate': 0.07353624922519299, 'subsample': 0.6783091201567526, 'colsample_bytree': 0.9075390032197173, 'scale_pos_weight': 4.4020893346839}. Best is trial 1 with value: 0.7882299249304652.
[I 2025-05-23 1

Best ROC-AUC: 0.7918337596873066
Best Parameters: {'n_estimators': 244, 'max_depth': 8, 'learning_rate': 0.02761997908148888, 'subsample': 0.6184627591588934, 'colsample_bytree': 0.5005091238340269, 'scale_pos_weight': 3.2313310760943366}


In [14]:
best_params = study.best_params
best_params.update({'tree_method': 'hist', 'use_label_encoder': False, 'random_state': 42})

final_model = ImbPipeline([
    ("preprocess", preprocessor),
    ("smote", SMOTE(random_state=42)),
    ("clf", XGBClassifier(**best_params))
])
final_model.fit(X, y)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['usia',
                                                   'jumlah_kontak_kampanye_ini',
                                                   'indeks_harga_konsumen',
                                                   'suku_bunga_euribor_3bln',
                                                   'rasio_sukses_sebelumnya',
                                                   'risk_score']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['pekerjaan',
                                                   'status_perkawinan',
                                                   'pendidikan', 'jenis_kontak',
                                                   'pulau', 'usia_group',
                                                   'pernah_...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.02761997908148888, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=8,
                               max_leaves=None, min_child_weight=None,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=244,
                               n_jobs=None, num_parallel_tree=None,
                               random_state=42, ...))])

In [15]:
val = pd.read_csv("validation_set.csv")
val = create_features(val)
X_val = val[features]
y_pred_prob = final_model.predict_proba(X_val)[:, 1]

if "berlangganan_deposito" in val.columns:
    y_val = val["berlangganan_deposito"]

    # ROC Curve
    fpr, tpr, _ = roc_curve(y_val, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(6, 4))
    plt.plot(fpr, tpr, label=f"ROC AUC = {roc_auc:.4f}")
    plt.plot([0, 1], [0, 1], "k--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend()
    plt.grid(True)
    plt.show()

    # PR Curve
    precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)
    plt.figure(figsize=(6, 4))
    plt.plot(recall, precision, label="PR Curve")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall Curve")
    plt.grid(True)
    plt.show()
else:
    print("Kolom 'berlangganan_deposito' tidak tersedia pada validation set. Melewati evaluasi ROC/PR.")

Kolom 'berlangganan_deposito' tidak tersedia pada validation set. Melewati evaluasi ROC/PR.


In [16]:
submission = pd.DataFrame({
    "customer_number": val["customer_number"],
    "berlangganan_deposito": y_pred_prob
})
submission.to_csv("submission.csv", index=False)
